In [ ]:
import pandas as pd
import numpy as np
import csv
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from pandas_profiling import ProfileReport
import warnings
warnings.filterwarnings('ignore')

In [ ]:
forest_fires_df=pd.read_csv("forest_fires_dataset.csv")
cols=['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH',
       'wind', 'rain', 'area']

In [ ]:
forest_fires_df.info()

Widzimy, że dane nie są wybrakowane, czyli nie mamy żadnych null'i.

In [ ]:
forest_fires_df['area'].describe()

Eksploracje zaczynamy od sprawdzenia informacji na temat 'area' czyli targetu, który chcielibyśmy modelować. Od razu widać, że większośc danych znajduje się w bliskiej odległości 0.

In [ ]:
sns.distplot(forest_fires_df['area'])
print("Skośność: %f" % forest_fires_df['area'].skew())
print("Kurtoza: %f" % forest_fires_df['area'].kurt())

Wykres, jak można się było spodziewać jest bardzo specyficzny.

In [ ]:
forest_fires_df['arealog'] = forest_fires_df['area'].map(lambda x: np.log(x+1))
fig, (ax1,ax2)=plt.subplots(1,2,figsize=(16, 8))
forest_fires_df['arealog'].plot.hist(ax=ax1)
sns.distplot(forest_fires_df['arealog'],ax=ax2,hist=False)
print("Skośność: %f" % forest_fires_df['arealog'].skew())
print("Kurtoza: %f" % forest_fires_df['arealog'].kurt())

Autorzy danych zasugerowali użycie log(x) na 'area', co wydaje się bardzo dobrym pomysłem do analizy tej danej.

In [ ]:
f, axs = plt.subplots(1,2,figsize=(14, 6))
ax=sns.boxplot(x='month', y="arealog", data=forest_fires_df,ax=axs[0],order=['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'])
plt.setp(ax.get_xticklabels(), rotation=45)
ax2=sns.boxplot(x='day', y="arealog", data=forest_fires_df,ax=axs[1],order=['mon','tue','wed','thu','fri','sat','sun'])

In [ ]:
f, axs = plt.subplots(2,1,figsize=(14, 6))
list_ordering = ['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'] 
forest_fires_df['month'] = pd.Categorical(forest_fires_df["month"], categories=list_ordering) 
sns.lineplot(x='month', y="arealog", data=forest_fires_df,ax=axs[0])

list_ordering = ['mon','tue','wed','thu','fri','sat','sun']
forest_fires_df['day'] = pd.Categorical(forest_fires_df["day"], categories=list_ordering) 
sns.lineplot(x='day', y="arealog", data=forest_fires_df,ax=axs[1])

Z boxplotów i lineplotów zrobionych na zmiennnych kategorycznych (miesiąc, dzień) nie widać jasnej zależności. Warto zauważyć relatywnie duży rozrzut w maju.

Warto jednak przyjżeć się jak rozkładają się cechy ciągłe, ponieważ już w tym miejscu może być widoczna jakaś zależność.

In [ ]:
forest_fires_df.hist(figsize=(18, 12))
plt.show()

Na pierwszy rzut oka można zacząć już coś podejrzewać, ale trzeba oczywiście to dokładnie zbadać i do tego posłuży nam heatmapa.

In [ ]:
corrmat=forest_fires_df[['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH','wind', 'rain', 'area','arealog']].corr()
k = 10
cols = corrmat.nlargest(k, 'arealog')['arealog'].index
cm = np.corrcoef(forest_fires_df[cols].values.T)
sns.set(font_scale=1.9)
f, ax = plt.subplots(figsize=(12, 9))
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 20}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

Pierwsza rzecz, którą widzimy to to, że ani 'area', ani 'arealog' nie jest wyraźnie mocniej skorelowana z którąkolwiek  z innych cech. Druga rzecz jest taka, że na heatmapie z naniesionymi wartościami korelacji, można już bardziej zdecydowanie wskazać cechy na które trzeba zwrócić uwagę. 

In [ ]:
sns.set()
cols=['FFMC', 'DMC', 'DC', 'ISI', 'temp', 'RH',
       'wind', 'rain']
sns.pairplot(forest_fires_df[cols])
plt.show();

Wiedząc już gdzie występują zależności możemy teraz spróbować znaleźć ich 'kształt' np. punkty na wykresie DC i DMC wyglądają jakby były skupione wokół jakiejś prostej, wykres FFMC i ISI wygląda podobnie do log(x).

In [ ]:
ProfileReport(forest_fires_df)

Takie narzędzie wydaje się bardzo przydatne w podstawowej analizie, ale na pewno nie jest wystarczające. Normalnie zobaczylibyśmy więcej wykresów takich jak boxploty i barploty. Podejrzewam również, że przy dużych zbiorach danych tworzenie takiego raportu może zająć znaczącą ilość czasu.